Previous: [pipeline_preprocess](pipeline_preprocess.ipynb)  ::: Next [process Monet](pipeline_prepare_monet.ipynb) 

---
## How visual stimuli are synchronized and matched
The stimulus computer and acquisition computers run on different clocks.  The table `preprocess.Sync` matches simultaneously recorded signals in both systems to synchronize the two recordings.

The field `frame_times` in `preprocess.Sync` contains the times of the two-photon frames on the same clock as the `flip_times` in `vis.Trial`.  Furthermore, it has the reference to the `vis.Session`'s `psy.id` and the trial ids of the first and last trial that were played during the given scan (`preprocess.Sync` has 1-to-1 correspondence scans). 

In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (6, 6)
import datajoint as dj
from pipeline import preprocess, vis


All kinds of stimuli are synchronized and matched exactly the same way using stimulus-neutral tables `preprocess.Sync`, `vis.Session`, `vis.Trial`, and `vis.Condition`.  All information pertaining to specific stimuli is stored in tables immediately belore `vis.Condition`.

In [ ]:
(dj.ERD(preprocess.Sync)-1+vis.Trial+(dj.ERD(vis.Condition)+1)).draw()

In [ ]:
preprocess.Sync().heading

In [ ]:
vis.Trial().heading

---

Let's pick an arbitrary scan that has Monet trials in it.

In [ ]:
scan_keys = list((preprocess.Sync() & vis.Monet()).fetch.keys())   # get the keys of all scans with Monet
key = scan_keys[10]   # let's pick an arbitrary scan
dict(key)

Joining `preprocess.Sync*vis.Trial` provides all the trials that were played for the given scan.  
Use the `first_trial` and `last_trial` fields to only include trials that played between the start time and the stop time of the scan.

In [ ]:
matched_trials = preprocess.Sync()*vis.Trial() & 'trial_idx between first_trial and last_trial' & key
matched_trials 

---
Now `matched_trials` contains almost all the information about the trials except the bitmaps stimulus conditions themselves, which are stored in vis.Monet and vis.MonetLookup()
Let's join them into the the same relation:

In [ ]:
matched_trials *= vis.Monet()*vis.MonetLookup()
matched_trials

Now `matched_trials` contains all the infromation about all the trials for the scan identify by `key`.
All that remains to be done is to fetch the contents.
```python
info = match_trials.fetch()   
```

---
Next [process Monet](pipeline_prepare_monet.ipynb) 